In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from torch.utils.data import DataLoader, Dataset
from torch import nn

In [ ]:
df = pd.read_parquet('Характеристики.parquet')

In [ ]:
contexts = df['context'].tolist()

In [ ]:
class ContextDataset(Dataset):
    def __init__(self, contexts):
        self.contexts = contexts

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        return self.contexts[idx]

dataset = ContextDataset(contexts)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
def contrastive_loss(embeddings, temperature=0.07):
    embeddings = nn.functional.normalize(embeddings, dim=1)
    similarity_matrix = torch.matmul(embeddings, embeddings.T)


    mask = torch.eye(similarity_matrix.shape[0], dtype=torch.bool, device=similarity_matrix.device)

    similarities = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)
    similarities = similarities / temperature

    targets = torch.arange(similarity_matrix.shape[0], device=similarity_matrix.device)
    loss = nn.CrossEntropyLoss()(similarities, targets)

    return loss

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        embeddings = model.encode(batch, convert_to_tensor=True)

        loss = contrastive_loss(embeddings)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

In [ ]:
model.save("labse_contrastive")